In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel("All_imputed_data.xlsx")

In [3]:
df_data = df.copy()

In [4]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
for i in df_data.columns:
    print(f"{i}:unique count -> {df_data[i].nunique()}, <> unique percentage -> {(df_data[i].nunique()/len(df_data))*100}% <>, null value percentage -> {(df_data[i].isnull().sum()/len(df_data))*100}%, Data types -> {df_data[i].dtypes}")
    print("-------------------------------------------------------------------------------------")

ft:unique count -> 4, <> unique percentage -> 0.055263885051119094% <>, null value percentage -> 0.0%, Data types -> object
-------------------------------------------------------------------------------------
bt:unique count -> 9, <> unique percentage -> 0.12434374136501797% <>, null value percentage -> 0.0%, Data types -> object
-------------------------------------------------------------------------------------
km:unique count -> 3579, <> unique percentage -> 49.44736114948881% <>, null value percentage -> 0.0%, Data types -> object
-------------------------------------------------------------------------------------
transmission:unique count -> 2, <> unique percentage -> 0.027631942525559547% <>, null value percentage -> 0.0%, Data types -> object
-------------------------------------------------------------------------------------
ownerno:unique count -> 5, <> unique percentage -> 0.06907985631389886% <>, null value percentage -> 0.0%, Data types -> int64
------------------------

In [5]:
df_data.drop(['owner'], axis=1, inplace=True)# Drop Duplicate column
df_data.select_dtypes(include='object').columns

Index(['ft', 'bt', 'km', 'transmission', 'oem', 'model', 'variantname',
       'price', 'registration_year', 'insurance_validity', 'seats', 'rto',
       'engine_displacement', 'max_power', 'torque', 'color', 'engine_type',
       'turbo_charger', 'super_charger', 'length', 'width', 'height',
       'wheel_base', 'gear_box', 'drive_type', 'steering_type',
       'front_brake_type', 'rear_brake_type', 'tyre_type', 'city'],
      dtype='object')

In [6]:
df_data.head(1)

,ft,bt,km,transmission,ownerno,oem,model,modelyear,centralvariantid,variantname,price,registration_year,insurance_validity,seats,rto,engine_displacement,year_of_manufacture,power_steering,power_windows_front,air_conditioner,heater,adjustable_head_lights,centeral_locking,child_safety_locks,power_windows_rear,remote_trunk_opener,remote_fuel_lid_opener,low_fuel_warning_light,accessory_power_outlet,rear_seat_headrest,glove_compartment,day_night_rear_view_mirror,passenger_side_rear_view_mirror,halogen_headlamps,rear_seat_belts,door_ajar_warning,anti_lock_braking_system,multifunction_steering_wheel,navigation_system,engine_start_stop_button,gear_shift_indicator,adjustable_steering,outside_temperature_display,rear_window_defogger,power_door_locks,driver_air_bag,passenger_air_bag,seat_belt_warning,engine_check_warning,crash_sensor,rear_camera,speed_sensing_auto_door_lock,pretensioners_and_force_limiter_seatbelts,impact_sensing_auto_door_lock,radio,integrated2din_audio,usb_auxiliary_input,bluetooth,touch_screen,driving_experience_control_eco,rear_reading_lamp,rear_acvents,air_quality_control,height_adjustable_front_seat_belts,cruise_control,voice_control,leather_seats,battery_saver,anti_theft_alarm,hill_assist,brake_assist,rain_sensing_wiper,adjustable_headrest,ledheadlights,cornering_headlamps,side_air_bag_front,side_air_bag_rear,tyre_pressure_monitor,clutch_lock,anti_pinch_power_windows,knee_airbags,wireless_phone_charging,projector_headlamps,speed_alert,rear_entertainment_system,ventilated_seats,view360camera,steering_mounted_tripmeter,hill_descent_control,roof_carrier,mileage,max_power,torque,color,engine_type,no_of_cylinder,values_per_cylinder,turbo_charger,super_charger,length,width,height,wheel_base,kerb_weight,gross_weight,gear_box,drive_type,steering_type,turning_radius,front_brake_type,rear_brake_type,top_speed,tyre_type,no_door_numbers,cargo_volumn,alloy_wheel_size,ground_clearance_unladen,city,stability_features,blind_spot_assistance,indicators,Adjustable_rear_view_mirror,fog_lamps,automatic_lamps,meters,car_roof,speakers,antenna,player,smartphone_integration,impact_beams
0,Petrol,Hatchback,"1,20,000",Manual,3,Maruti,Maruti Celerio,2015,3979,VXI,₹ 4 Lakh,2015,Third Party insurance,5 Seats,KA51,998 cc,2015,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,23.1,67.04bhp@6000rpm,90Nm,White,K10B Engine,3,4,no,no,3715mm,1635mm,1565mm,2425mm,835.0,1197.110232,5 Speed,FWD,Power,4.7,Ventilated Disc,Drum,150.0,"Tubeless, Radial",5,235,14,165,bangalore,False,False,False,True,False,False,True,False,False,True,False,False,True


In [7]:
df_data[['km', 'price', 'registration_year','seats', 'engine_displacement', 'max_power', 'torque', 'length', 'width','height', 'wheel_base', 'gear_box']].head(1)

,km,price,registration_year,seats,engine_displacement,max_power,torque,length,width,height,wheel_base,gear_box
0,"1,20,000",₹ 4 Lakh,2015,5 Seats,998 cc,67.04bhp@6000rpm,90Nm,3715mm,1635mm,1565mm,2425mm,5 Speed


In [8]:
df_data['km'] = df_data['km'].astype(str).str.lower()
df_data['km'] = df_data['km'].str.replace(r'[^0-9]', '', regex=True)
df_data['km'] = pd.to_numeric(df_data['km'], errors='coerce')

In [9]:
def convert_price(price_str):
    # Remove the currency symbol and commas
    price_str = re.sub(r'[₹,]', '', price_str).strip()
    
    # Handle different units
    if 'Crore' in price_str:
        price_str = price_str.replace('Crore', '').strip()
        return float(price_str) * 10000000  # Convert to the base unit (e.g., rupees)
    elif 'Lakh' in price_str:
        price_str = price_str.replace('Lakh', '').strip()
        return float(price_str) * 100000  # Convert to the base unit (e.g., rupees)
    else:
        return float(price_str)  # Assume it's already in the base unit (e.g., rupees)

# Apply the conversion function to the 'price' column
df_data['price'] = df_data['price'].apply(convert_price)
df_data['price'].unique()

array([ 400000.,  811000.,  585000., ..., 5541000.,  158000.,   35000.])

In [10]:
# Extract numeric year values from 'registration_year' column and convert to numeric, coercing errors to NaN
df_data['registration_year'] = df_data['registration_year'].str.extract(r'(\d{4})', expand=False)
df_data['registration_year'] = pd.to_numeric(df_data['registration_year'], errors='coerce')

# Remove " Seats" suffix from 'seats' column and convert to numeric, coercing errors to NaN
df_data['seats'] = df_data['seats'].str.replace(" Seats", "", regex=False)
df_data['seats'] = pd.to_numeric(df_data['seats'], errors='coerce')

# Clean and convert 'length', 'width', 'height', and 'wheel_base' columns by removing non-numeric characters and converting to numeric
dimensions = ['length', 'width', 'height', 'wheel_base']
for column in dimensions:
    df_data[column] = df_data[column].str.replace(r'[^0-9]', '', regex=True)
    df_data[column] = pd.to_numeric(df_data[column], errors='coerce')

# Correct specific erroneous value in 'height' column
df_data['height'] = df_data['height'].replace(14981501, 1501)

In [12]:
# Define a dictionary with patterns and their replacements
torque_replacements = {
    "190 Nm at 1750 rpm": "190",
    "190 Nm at 1750  rpm": "190",
    "110 Nm at 3750 rpm": "110",
    "210/1750-4600": "210",
    "110 (11.2)": "110",
    "153 Nm at 3800 rpm": "153",
    "210 / 1900": "210",
    "180 Nm at 1440-1500rpm": "180",
    "22.4 kgm at 1750-2750rpm": "22.4kgm",
    "146Nm at 4800 rpm": "146",
    "260 Nm at 1800-2200 rpm": "260",
    "14.9 KGM at 3000 RPM": "14.9kgm",
    "11.4 kgm at 4,000 rpm": "11.4kgm",
    "155 nm at 1600-2800 rpm": "155",
    "155  at 1600-2800 rpm": "155"
}

# Replace patterns based on the dictionary
for pattern, replacement in torque_replacements.items():
    df_data['torque'] = df_data['torque'].str.replace(pattern, replacement, regex=False)

In [13]:
df_data['torque'] = df_data['torque'].str.replace("155  at 1600-2800 rpm", "155", regex=False)

# Convert all values in 'torque' column to lowercase
df_data['torque'] = df_data['torque'].astype(str).str.lower()

# Define the conversion function
def convert_torque(torque_str):
    # Remove commas and extra spaces but keep the units (e.g., "nm", "kgm", "kg")
    torque_str = re.sub(r'[,\s]+', ' ', torque_str).strip()
    
    # Handle different units by checking for "nm", "kgm", or "kg"
    if 'nm' in torque_str:
        torque_value = re.sub(r'nm', '', torque_str).strip()
        return float(torque_value)
    elif 'kgm' in torque_str:
        torque_value = re.sub(r'kgm', '', torque_str).strip()
        return float(torque_value) * 9.8
    elif 'kg' in torque_str:
        torque_value = re.sub(r'kg', '', torque_str).strip()
        return float(torque_value) * 9.8
    else:
        try:
            return float(torque_str)
        except ValueError:
            return None  # Return None if conversion fails

# Apply the function to the 'torque' column
df_data['torque'] = df_data['torque'].apply(convert_torque)

# Print the unique values after conversion
print(df_data['torque'].dtypes)

float64


In [14]:
df_data['gear_box'] = df_data['gear_box'].astype(str).str.lower()
df_data['gears'] = df_data['gear_box']
df_data.drop(['gear_box'], axis=1, inplace=True)

In [15]:
# Define a mapping of text patterns to replace with gear numbers
gear_replacements = {
    "ecvt": "6",
    "single speed": "1",
    "single speed reduction gear": "1",
    "1 reverse": "5",
    "six speed  gearbox": "6",
    "single-speed transmission": "1",
    "fully automatic": "6",
    "six speed automatic gearbox": "6",
    "six speed geartronic, six speed automati": "6",
    "six speed automatic transmission": "6",
    "six speed manual with paddle shifter": "6",
    "e-cvt": "6",
    "five speed manual transmission gearbox": "5",
    "six speed manual": "6",
    "five speed": "5",
    "ags": "6",
    "imt": "6",
    "five speed manual transmission": "5",
    "ivt": "6",
    "direct drive": "6",
    "7g dct 7-speed dual clutch transmission": "7",
    "five speed manual": "5",
    "7 speed 7g-dct": "7",
    "8 speed 6": "8",
    "5 speed 6": "5",
    "5 speed forward, 5": "5",
    "7 speed 6": "7",
    "6-speed 6": "6",
    "6 speed 6": "6",
    "cvt": "6"
}

# Apply the replacements using a loop
for old_value, new_value in gear_replacements.items():
    df_data['gears'] = df_data['gears'].str.replace(old_value, new_value, regex=False)

# Remove any remaining non-numeric characters and convert to numeric
df_data['gears'] = df_data['gears'].str.replace(r'[^0-9]', '', regex=True)
df_data['gears'] = pd.to_numeric(df_data['gears'], errors='coerce')

In [16]:
df_data['max_power'] = df_data['max_power'].astype(str).str.lower()
df_data['max_power_bhp'] = (
    df_data['max_power']
    .str.split('@|bhp|ps|\(|\[|hp')
    .str[0]
    .str.replace(r'[^0-9.]', '', regex=True)
    .astype(float, errors='ignore')
)

In [17]:
df_data.select_dtypes(include='object').shape[1]

20

In [18]:
# Load the dataset
df1 = pd.read_excel("combined_data.xlsx")

# Create a dictionary to map unique engine_displacement values to Displacement
displacement = dict(zip(df1['Engine Displacement'], df1['Displacement']))

# Map the engine_displacement values in df_data using the displacement dictionary
df_data['engine_displacement'] = df_data['engine_displacement'].map(displacement)

In [19]:
for column in df_data.select_dtypes(include='object').columns:
    df_data[column] = df_data[column].astype(str).str.lower()

In [20]:
df_data['insurance_validity'].unique()

array(['third party insurance', 'comprehensive', 'third party',
       'zero dep', '2', '1', 'not available'], dtype=object)

In [21]:
def update_insurance_validity(value):

    if 'third party' in value:
        return 'third party'
    elif 'comprehensive' in value or 'zero dep' in value:
        return value
    else:
        return 'not available'

df_data['insurance_validity'] = df_data['insurance_validity'].apply(update_insurance_validity)

In [22]:
for value in ['epas', 'electrical', 'electronic','electric']:
    df_data['steering_type'] = df_data['steering_type'].str.replace(value, "power", regex=False)

In [23]:
def update_brake(value):

    if 'drum' in value:
        return 'drum'
    elif 'disc' in value or 'disk' in value or 'abs' in value or 'electric parking brake' in value:
        return 'disc'
    elif 'disc & drum' in value or 'drum in disc' in value:
        return 'disc'
    else:
        return 'drum'

df_data['front_brake_type_group'] = df_data['front_brake_type'].apply(update_brake)
df_data['rear_brake_type_group'] = df_data['rear_brake_type'].apply(update_brake)

In [24]:
def update_tyre_type(tyre_type_str):

    if 'tubeless' in tyre_type_str and 'radial' in tyre_type_str:
        return 'radial tubeless'
    elif 'tubless' in tyre_type_str and 'runflat' in tyre_type_str:
        return 'tubeless and runflat'
    elif 'tubless' in tyre_type_str and 'radial' in tyre_type_str:
        return 'radial tubeless'
    elif 'tubeless' in tyre_type_str and 'runflat' in tyre_type_str:
        return 'tubeless and runflat'
    elif 'radial' in tyre_type_str and 'runflat' in tyre_type_str:
        return 'radial and runflat'
    elif 'radial' in tyre_type_str:
        return 'radial'
    elif 'runflat' in tyre_type_str:
        return 'runflat'
    else:
        return 'tubeless'

df_data['tyre_type_group'] = df_data['tyre_type'].apply(update_tyre_type)

In [25]:
def determine_color(color):
    
    if 'white' in color:
        return 'White'
    elif 'red' in color or 'wine' in color or 'cherry' in color:
        return 'Red'
    elif 'blue' in color:
        return 'Blue'
    elif 'brown' in color:
        return 'Brown'
    elif 'gold' in color:
        return 'Gold'
    elif 'green' in color:
        return 'Green'
    elif 'black' in color:
        return 'Black'     
    elif 'orange' in color:
        return 'Orange'
    elif 'yellow' in color:
        return 'Yellow'
    elif 'bronze' in color:
        return 'Bronze'
    elif 'purple' in color or 'violet' in color:
        return 'Purple'
    elif 'grey' in color or 'gray' in color:
        return 'Gray'
    elif 'silver' in color or 'metal' in color or 'steel' in color:
        return 'Silver'
    else:
        return 'Other'

# Apply the function to the DataFrame column
df_data['color_group'] = df_data['color'].apply(determine_color)
df_data.drop(['max_power','front_brake_type','rear_brake_type','color','tyre_type'], axis=1, inplace=True)

In [26]:
df_data['color_group'].value_counts()

color_group
Other     2001
White     1465
Silver    1148
Red        722
Gray       574
Blue       522
Black      333
Brown      256
Orange      96
Gold        54
Green       33
Yellow      17
Purple      13
Bronze       4
Name: count, dtype: int64

In [27]:
df_data.to_excel("Standarded_Data_Format.xlsx",index=False)